In [1]:
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.2 MB/s eta 0:00:00


In [2]:
from langchain.chains import LLMChain,SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate

from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap
import sys

In [3]:
#tokenizer = AutoTokenizer.from_pretrained("TinyPixel/Llama-2-7B-bf16-sharded")
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [4]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [6]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """You are tasked with educating users about solar power generation and various aspects of renewable and clean energy. Your responses should be informative, accurate, and geared towards promoting sustainable and eco-friendly practices. Please ensure that the information you provide is accurate and backed by credible sources. Keep your responses clear and concise, explaining complex concepts in a way that non-experts can understand. When discussing topics, try to explain not only what is being done but why and how. Encourage users to adopt more sustainable habits and technologies, explaining the environmental and economic benefits of renewable energy sources to inspire positive change. Maintain a neutral stance, avoiding endorsements or criticisms of specific brands, products, or political viewpoints. Ensure that the information you provide adheres to safety guidelines and ethical considerations, prioritizing the well-being of individuals and the environment. After providing an initial response, ask users if they have more questions or need further clarification, and be responsive to user inquiries."""

In [7]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')




def model_to_user(lang, text):
      translator = Translator()
      translated_text = translator.translate(text, src='en', dest=lang)
      return f"{translated_text.text}"

# def user_to_model(lang,text):
#       translator = Translator()
#       translated_text = translator.translate(text, src=lang, dest='en')
#       return translated_text.text

In [8]:

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 1000, 'top_k' :200})



In [9]:
system_prompt = """You are tasked with educating users about solar power generation and various aspects of renewable and clean energy.
Your responses should be informative, accurate, and geared towards promoting sustainable and eco-friendly practices. Please ensure that the information you provide
is accurate and backed by credible sources. Keep your responses clear and concise, explaining complex concepts in a way that non-experts can understand. When discussing topics,
try to explain not only what is being done but why and how. Encourage users to adopt more sustainable habits and technologies, explaining the environmental and economic benefits of
renewable energy sources to inspire positive change. Maintain a neutral stance, avoiding endorsements or criticisms of specific brands, products, or political viewpoints.
Ensure that the information you provide adheres to safety guidelines and ethical considerations, prioritizing the well-being of individuals and the environment.
After providing an initial response, ask users if they have more questions or need further clarification, and be responsive to user inquiries."""
instruction = "Answer accurately based on the question: \n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)


[INST]<>
You are tasked with educating users about solar power generation and various aspects of renewable and clean energy.
Your responses should be informative, accurate, and geared towards promoting sustainable and eco-friendly practices. Please ensure that the information you provide
is accurate and backed by credible sources. Keep your responses clear and concise, explaining complex concepts in a way that non-experts can understand. When discussing topics,
try to explain not only what is being done but why and how. Encourage users to adopt more sustainable habits and technologies, explaining the environmental and economic benefits of
renewable energy sources to inspire positive change. Maintain a neutral stance, avoiding endorsements or criticisms of specific brands, products, or political viewpoints.
Ensure that the information you provide adheres to safety guidelines and ethical considerations, prioritizing the well-being of individuals and the environment.
After providing an in

In [10]:
prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)

In [11]:
text = "why is my power output so low??"

In [12]:
response = llm_chain.run(text)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


  Thank you for reaching out! I'm here to help you understand why your power output might be lower than expected.

There could be several reasons causing your low power output, including:

1. Insufficient solar panel angle: Make sure your solar panels are facing directly at the sun's rays to maximize energy production. If they're not at the optimal angle, they might not be efficient enough to generate enough power.
2. Dirt, debris, or shading: Clean your solar panels regularly to ensure they're working at their best. If there are any obstructions or shading from trees, buildings, or other objects nearby, it could reduce the amount of sunlight reaching the panels.
3. Inverter issues: Your inverter converts the DC power generated by your solar panels into AC power for your home. If the inverter is malfunctioning or not calibrated properly, it could result in lower power output.
4. Wiring issues: Check your solar panel connections and wiring to ensure everything is secure and not damaged.

In [13]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [14]:
!pip install flask-ngrok

In [15]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=36de1b31192485b2ee7d7cb05606aa9e57f0c883f7d2cc1fcdf4b9b48bc1a7c0
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [16]:
!ngrok config add-authtoken 2VCFw0HKXnTygcSKqVlrNfmy7MA_522AWhDWGypkY4bXeQccM

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [17]:
!pip install -U flask-cors

In [19]:
from flask import Flask,jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
import logging
app = Flask(__name__)
logging.basicConfig(level=logging.INFO)
logging.getLogger('flask_cors').level = logging.DEBUG
CORS(app)
run_with_ngrok(app)


@app.route("/get/<querry>", methods = ["GET"])
def home(querry=None):
  try:
    response = llm_chain.run(querry)
    final = {
      "user_query": text,
      "Response": response
  }
    return jsonify(final)
  except:
    return {"error":"error"}

app.run()


DEBUG:flask_cors.extension:Configuring CORS with resources: {'/*': {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}}


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://73ec-34-145-29-28.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


DEBUG:flask_cors.extension:Request to '/' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:Settings CORS headers: MultiDict([('Access-Control-Allow-Origin', '*')])
DEBUG:flask_cors.extension:CORS have been already evaluated, skipping
INFO:werkzeug:127.0.0.1 - - [28/Oct/2023 09:07:48] "GET / HTTP/1.1" 404 -
DEBUG:flask_cors.extension:Request to '/get/My solar panel is not working what can be the issues' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_option